# Шаги mapreduce для карт
Предварительные наброски кода.

- Deck – колода
- Suit – масть
- Hearts – червы
- Diamonds – бубны
- Clubs – трефы
- Spades – пики
- Jack – валет
- Queen – дама
- King – король
- Ace – туз
- Joker – джокер

## Map
Запишем поток строк в более удобном виде и оставим все значения больше шестерки.

In [1]:
with open('cards.csv') as f:
    for i, line in enumerate(f):
        suit, value = line.strip().split(',')
        print(suit, value)
        
        if i > 5:
            break

червы 4
пики 4
червы 10
бубны 10
червы 5
пики 9
трефы дама


In [2]:
SUITES = {
    'червы': 'H',
    'пики': 'S',
    'бубны': 'D',
    'трефы': 'C',
}

VALUES = {
    'валет': 'J',
    'дама': 'Q',
    'король': 'K',
    'туз': 'A',
}

In [3]:
with open('cards.csv') as f:
    for i, line in enumerate(f):
        suit, value = line.strip().split(',')
        
        suit = SUITES[suit]
        
        if value in VALUES:
            value = VALUES[value]
        
        print(f'{suit},{value},1')
        
        if i > 5:
            break

H,4,1
S,4,1
H,10,1
D,10,1
H,5,1
S,9,1
C,Q,1


In [4]:
with open('cards_after_mapper.csv', 'w') as f_mapper:
    with open('cards.csv') as f:
        for i, line in enumerate(f):
            suit, value = line.strip().split(',')

            suit = SUITES[suit]

            if value in VALUES:
                value = VALUES[value]

            if value.isdigit() and int(value) >= 6:
                f_mapper.write(f'{suit},{value},1\n')

## Shuffle
```
cat cards_after_mapper.csv | sort > cards_sorted.csv
```

## Reduce
Сначала посчитаем количество мастей карт

In [5]:
with open('cards_sorted.csv') as f:
    for i, line in enumerate(f):
        suit, value, one = line.strip().split(',')
        print(suit, value, one)
        
        if i > 5:
            break

C 10 1
C 10 1
C 10 1
C 10 1
C 10 1
C 10 1
C 10 1


In [6]:
previous_suit = None
suit_count = 0

with open('cards_sorted.csv') as f:
    for i, line in enumerate(f):
        suit, value, one = line.strip().split(',')
        one = int(one)
        
        if previous_suit:
            if previous_suit == suit:
                suit_count += one
            else:
                print(previous_suit, suit_count)
                suit_count = one
                previous_suit = suit
        else:
            previous_suit = suit
            suit_count = one
        
print(previous_suit, suit_count)

C 8990
D 8918
H 8920
S 8929


Уберем дубликат строчки
```python
previous_suit = suit
```

In [7]:
previous_suit = None
suit_count = 0

with open('cards_sorted.csv') as f:
    for i, line in enumerate(f):
        suit, value, one = line.strip().split(',')
        one = int(one)
        
        if previous_suit:
            if previous_suit == suit:
                suit_count += one
            else:
                print(previous_suit, suit_count)
                suit_count = one   
        else:
            suit_count = one
        
        previous_suit = suit
        
print(previous_suit, suit_count)

C 8990
D 8918
H 8920
S 8929


## Reduce v2
Посчитаем статистику для пар масть + значение

In [8]:
previous_suit = None
previous_value = None

count = 0

with open('cards_sorted.csv') as f:
    for i, line in enumerate(f):
        suit, value, one = line.strip().split(',')
        one = int(one)
        
        if previous_suit:
            if previous_suit == suit and previous_value == value:
                count += one
            else:
                print(previous_suit, previous_value, count)
                
                previous_suit = suit
                previous_value = value
                count = one
                
        else:
            previous_suit = suit
            previous_value = value
            count = one
            
print(previous_suit, previous_value, count)

C 10 1834
C 6 1763
C 7 1806
C 8 1779
C 9 1808
D 10 1847
D 6 1743
D 7 1756
D 8 1769
D 9 1803
H 10 1816
H 6 1739
H 7 1811
H 8 1739
H 9 1815
S 10 1749
S 6 1803
S 7 1795
S 8 1764
S 9 1818
